## Publish Regular 2D Grid and Colormap objects

This example shows:

- How to convert a 2D grid in a `grd` file into an Evo geoscience object using the Geosoft Python SDK and the Evo Python SDK.
- How to convert a colormap in a `grd` file into an Evo colormap object and associate it with a geoscience object using the Evo Python SDK and the Evo Colormap API.

### Requirements

You must have a Seequent account with the Evo entitlement to use this notebook.

The following parameters must be provided:

- The client ID of your Evo application.
- The callback/redirect URL of your Evo application.

To obtain these app credentials, refer to the [Apps and tokens guide](https://developer.seequent.com/docs/guides/getting-started/apps-and-tokens) in the Seequent Developer Portal.

In [ ]:
import uuid

import geosoft.gxpy.grid as gxgrid
import geosoft.gxpy.gx as gx
import pandas as pd
import requests
from evo_schemas.components import (
    AttributeDescription_V1_0_1,
    BoundingBox_V1_0_1,
    ContinuousAttribute_V1_1_0,
    Crs_V1_0_1_EpsgCode,
    Crs_V1_0_1_OgcWkt,
    NanContinuous_V1_0_1,
    Rotation_V1_1_0,
)
from evo_schemas.elements import FloatArray1_V1_0_1
from evo_schemas.objects import Regular2DGrid_V1_2_0
from geosoft.gxapi import GXAPIError
from IPython.display import HTML, Image, display

from evo.notebooks import FeedbackWidget, ServiceManagerWidget
from evo.objects import ObjectAPIClient

# Create a GX context
try:
    gxc = gx.GXpy()
except GXAPIError as e:
    if str(e) == "A GXpy instance has already been created for current thread.":
        pass
    else:
        raise RuntimeError(f"GXAPIError occurred: {e}")

# Define the cache location and input path.
cache_location = "data"
input_path = f"{cache_location}/input"

# Define input file path.
input_file = f"{input_path}/Magnetics.grd"

# Evo app credentials
client_id = "<your-client-id>"
redirect_url = "<your-redirect-url>"

manager = await ServiceManagerWidget.with_auth_code(
    discovery_url="https://discover.api.seequent.com",
    redirect_url=redirect_url,
    client_id=client_id,
    cache_location=cache_location,
).login()

### Use the Evo Python SDK to create an object client and a data client

In [ ]:
# The object client will manage your auth token and Geoscience Object API requests.
object_client = ObjectAPIClient(manager.get_environment(), manager.get_connector())

# The data client will manage saving your data as Parquet and publishing your data to Evo storage.
data_client = object_client.get_data_client(manager.cache)

### Define helper functions

These functions assist with converting and publishing colormaps to Evo, and for helping the user to view their new object in the Evo portal.

In [ ]:
def convert_colormap(colormap, min_value, max_value):
    """
    Converts the colormap to a format suitable for uploading to Evo.

    Args:
        colormap (list): A list of tuples where each tuple contains an attribute value and its corresponding RGB color.
        min_value (float): The minimum value of the data range.
        max_value (float): The maximum value of the data range.

    Returns:
        dict: A dictionary containing the converted colormap data, including:
            - name (str): The name of the colormap.
            - schema (str): The schema type, e.g., "continuous".
            - attribute_controls (list): A list of attribute values corresponding to the colormap.
            - colors (list): A list of RGB color values.
            - gradient_controls (list): A list of normalized gradient control values.
    """
    # Convert the colormap to a format suitable for Evo
    # This is a placeholder implementation; you may need to adjust it based on your requirements
    # num_colors = len(colormap)
    attribute_controls = []
    rgb_colors = []

    # Ensure the first attribute control matches the minimum value
    if colormap[0][0] != min_value:
        attribute_controls.append(min_value)
        rgb_colors.append(colormap[0][1])

    # Extract attribute controls and RGB colors from the colormap
    for attribute_value, color in colormap:
        if attribute_value is not None and color is not None:
            attribute_controls.append(attribute_value)
            rgb_colors.append(color)

    # Ensure the last attribute control matches the maximum value
    if colormap[-1][0] != max_value:
        attribute_controls.append(max_value)
        rgb_colors.append(colormap[-1][1])

    # Normalize gradient controls
    gradient_controls = [float(i) / (len(attribute_controls) - 1) for i in range(len(attribute_controls))]

    colormap_data = {
        "name": "custom_colormap",
        "schema": "continuous",
        "attribute_controls": attribute_controls,
        "colors": rgb_colors,
        "gradient_controls": gradient_controls,
    }

    return colormap_data


def publish_colormap(grid, object_metadata, http_headers):
    """
    Converts and uploads the colormap to an Evo workspace.

    Args:
        grid (geosoft.gxpy.grid.Grid): The grid object containing the data and colormap.
        object_metadata (evo.objects.data.ObjectMetadata): Metadata of the object associated with the colormap.
        http_headers (dict): HTTP headers, including the authorization token.

    Returns:
        dict: Metadata of the uploaded colormap, or None if the upload fails.
    """
    auth_header = {"Authorization": http_headers.get("Authorization")}
    hub_url = object_metadata.environment.hub_url
    org_id = object_metadata.environment.org_id
    workspace_id = object_metadata.environment.workspace_id
    url = f"{hub_url}/colormap/orgs/{org_id}/workspaces/{workspace_id}/colormaps"

    # Extract the data values from the grid and flatten them into a 1D array.
    values = grid.xyzv()[:, :, 3]
    flattened_values = values.flatten(order="C")
    values_df = pd.DataFrame(flattened_values, columns=["data"])

    # Find the minimum and maximum values in the data.
    min_value = values_df["data"].min()
    max_value = values_df["data"].max()

    # Extract the colormap from the grid
    # The colormap is a list of tuples where each tuple contains an attribute value and its corresponding RGB color.
    colormap = grid.get_default_color_map().color_map_rgb

    # Convert the Geosoft colormap to an Evo colormap.
    colormap_data = convert_colormap(colormap, min_value, max_value)

    # Upload the colormap to Evo.
    response = requests.post(
        url=url,
        json=colormap_data,
        headers=auth_header,
    )

    colormap_name = colormap_data.get("name")

    if response.status_code == 201:
        colormap_metadata = response.json()
        print(f"Colormap '{colormap_name}' created.")
        return colormap_metadata
    else:
        print(f"Failed to create colormap '{colormap_name}': {response.text}")
        return None


def associate_colormap_with_object(colormap_metadata, object_metadata, attribute_key, http_headers):
    """
    Associates the uploaded colormap with the object in the Evo platform.

    This function establishes a link between the colormap and the object by
    associating the colormap metadata with the object metadata using the
    specified attribute key.

    Args:
        colormap_metadata (dict): Metadata of the uploaded colormap, including its ID and other details.
        object_metadata (evo.objects.data.ObjectMetadata): Metadata of the object to associate the colormap with.
        attribute_key (str): The key of the attribute to associate the colormap with.
        http_headers (dict): HTTP headers, including the authorization token.

    Returns:
        None
    """

    auth_header = {"Authorization": http_headers.get("Authorization")}
    hub_url = object_metadata.environment.hub_url
    org_id = object_metadata.environment.org_id
    workspace_id = object_metadata.environment.workspace_id
    object_id = object_metadata.id
    url = f"{hub_url}/colormap/orgs/{org_id}/workspaces/{workspace_id}/objects/{object_id}/associations"

    # Prepare the colormap + geoscience object association payload
    association_data = {
        "attribute_id": attribute_key,
        "colormap_id": colormap_metadata["id"],
    }

    # Associate the colormap with the object
    response = requests.post(
        url,
        json=association_data,
        headers=auth_header,
    )

    colormap_name = colormap_metadata.get("name")
    object_name = object_metadata.name
    if response.status_code == 200:
        print(f"Colormap '{colormap_name}' associated with object '{object_name}'.")
    else:
        print("Failed to associate colormap:", response.text)


def build_portal_url(object_metadata):
    """
    Build a URL to take the user directly to the 3D visualization of a geoscience object in the Evo Portal.

    Args:
        object_metadata (evo.objects.data.ObjectMetadata): The metadata of the object for which the URL is being built.

    Returns:
        None: The function displays an HTML link to the Evo Portal for the specified object.
    """

    # Get the environment details from the object metadata
    hub_url = object_metadata.environment.hub_url
    hub_code = hub_url.split("://")[1].split(".")[0]
    org_id = object_metadata.environment.org_id
    workspace_id = object_metadata.environment.workspace_id
    object_id = object_metadata.id

    url = f"https://evo.seequent.com/{org_id}/workspaces/{hub_code}/{workspace_id}/viewer?id={object_id}"

    display(HTML(f'<a href="{url}" target="_blank">View object in the Evo Portal</a>'))

### Define object metadata

Geoscience object data must conform to a specific object schema. The `evo-schemas` package provides Pydantic models that make it easy to work with the equivalent JSON schemas. 
For this example we'll use v1.2.0 of the regular 2D grid schema, via the relevant Pydantic model.

Enter values for these parameters that are required by the object schema.
- `object_path`: The file path where the object will be found.
- `object_tags`: (Optional) A dictionary of additional tags to be assigned to the object. Leave as `None` is not required.

The name and coordinate reference system of the object will be extracted from the grid file.

In [ ]:
object_path = "Jupyter_Example"
object_tags = {"Source": "Jupyter Notebook"}

### Define object attributes and keys

In [ ]:
# List all of the attributes to be included in the object. Every attribute must have a unique key associated with it.
# Keys must be unique across the entire object, and we recommend saving a reference to the keys for later use.

object_attributes = {"2d-grid-data-continuous": str(uuid.uuid4())}

In [ ]:
# Display the grid.
with gxgrid.Grid.open(input_file) as grid_data:
    image_file = grid_data.image_file(shade=False, pix_width=500)

Image(image_file)

In [ ]:
# Define object name, origin, coordinate system, rotation, cell size and overall size of the grid.

with gxgrid.Grid.open(input_file) as grid_data:
    # Find the grid name. The new Evo object will be named after the grid.
    object_name = grid_data.name

    # Find the EPSG code and create a `Crs_V1_0_1_EpsgCode`` component.
    # If the grid file doesn't contain a coordinate system, create a `Crs_V1_0_1_OgcWkt` component and set the CRS to unspecified.
    try:
        epsg_code = int(grid_data.coordinate_system.esri_wkt.split('AUTHORITY["EPSG",')[1].split("]")[0])
    except IndexError:
        print("EPSG code not found. Setting the coordinate system to 'unspecified'.")
        coordinate_reference_system = Crs_V1_0_1_OgcWkt(ogc_wkt="unspecified")
    else:
        coordinate_reference_system = Crs_V1_0_1_EpsgCode(epsg_code=epsg_code)

    # Find the shape and size of the data.
    data_shape = grid_data.xyzv().shape
    size_x = data_shape[1]
    size_y = data_shape[0]
    size = [size_x, size_y]

    # Find the cell size and origin point.
    dx = grid_data.dx
    dy = grid_data.dy
    cell_size = [dx, dy]
    origin = [grid_data.x0, grid_data.y0, grid_data.extent_xyz[2]]

    # Find the rotation and create a `Rotation_V1_1_0` component.
    dip_azimuth = grid_data.rot
    rotation = Rotation_V1_1_0(dip=0.0, dip_azimuth=dip_azimuth, pitch=0.0)

    # Find the object extents and create a `BoundingBox_V1_0_1` component.
    bounding_box = BoundingBox_V1_0_1(
        min_x=grid_data.extent_xyz[0],
        min_y=grid_data.extent_xyz[1],
        min_z=grid_data.extent_xyz[2],
        max_x=grid_data.extent_xyz[3],
        max_y=grid_data.extent_xyz[4],
        max_z=grid_data.extent_xyz[5],
    )

In [ ]:
# Define the cell attributes.

# Create an empty list to store the cell attributes.
cell_attributes = []

# Create an empty dictionary to map file hashes to corresponding file paths.
output_file_hashes = {}

# 2d-grid-data-continuous
name = "2d-grid-data-continuous"
with gxgrid.Grid.open(input_file) as grid:
    # Extract the data values from the grid and flatten them into a 1D array.
    values = grid.xyzv()[:, :, 3]
    flattened_values = values.flatten(order="C")
    values_df = pd.DataFrame(flattened_values, columns=["data"])

    # Use the data client to save the DataFrame as a Parquet file and get the file hash.
    values_pq = FloatArray1_V1_0_1.from_dict(data_client.save_dataframe(values_df))

    # Optional: Create an `AttributeDescription_V1_0_1` component to describe the attribute.
    attribute_description = AttributeDescription_V1_0_1(
        type="Mag_M0",
        unit="nT",
        discipline="Geophysical",
        tags={"Source": "Jupyter Notebook"},
    )

    # Create a `ContinuousAttribute_V1_1_0` component to combine all the previous components.
    attribute = ContinuousAttribute_V1_1_0(
        name=name,
        key=object_attributes[name],
        nan_description=NanContinuous_V1_0_1(values=[]),
        values=values_pq,
        attribute_description=attribute_description,
    )

    # Add the attribute to the cell attributes list.
    cell_attributes.append(attribute)

### Create a new regular 2D grid object and publish it to Evo

In [ ]:
# Assemble the complete geoscience object by combining all previously defined components.
# - The name and UUID are used to identify the object.
# - The UUID is set to None because this is a new object. A new UUID will be assigned by the Evo service.
# - The bounding box defines the spatial extent of the object.
# - The tags provide metadata about the object.
# - The coordinate reference system defines the spatial reference for the object.
# - The remaining parameters define the grid properties.

grid = Regular2DGrid_V1_2_0(
    name=object_name,
    uuid=None,
    bounding_box=bounding_box,
    tags=object_tags,
    coordinate_reference_system=coordinate_reference_system,
    origin=origin,
    size=size,
    cell_size=cell_size,
    rotation=rotation,
    cell_attributes=cell_attributes,
)

# Upload the Parquet data to Evo.
await data_client.upload_referenced_data(grid.as_dict(), FeedbackWidget("Uploading data"))

# Define the object path.
full_obj_path = f"{object_path}/{object_name}.json"

# Create the geoscience object.
new_grid_metadata = await object_client.create_geoscience_object(full_obj_path, grid.as_dict())

### Create the colormap

In [ ]:
# Extract the http authorization headers from the connector.
connector = manager.get_connector()
http_headers = await connector._authorizer.get_default_headers()

with gxgrid.Grid.open(input_file) as grid:
    new_colormap_metadata = publish_colormap(grid, new_grid_metadata, http_headers)

    if new_colormap_metadata is not None:
        # Associate the colormap with the object.
        attribute_key = object_attributes["2d-grid-data-continuous"]
        associate_colormap_with_object(new_colormap_metadata, new_grid_metadata, attribute_key, http_headers)

### View the object in the Evo portal

In [ ]:
build_portal_url(new_grid_metadata)

Success! You now have a new geoscience object in Evo containing your grid data.

## Summary

In this example, we've completed the following:
* Analysed the grid file and constructed the elements and components required for a regular 2D grid.
* Converted the input data into Parquet format and saved it to the local cache.
* Combined all of the elements, components and data references into the `regular-2d-grid` schema format.
* Uploaded the Parquet files and the newly assembled object in JSON format to Evo.
* Analysed the colormap from the grid file and constructed an equivalent colormap in the Evo format.
* Uploaded the new colormap to Evo and associated it with the new grid.